---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # For each of the five graphs, assign the label 'PA', 'SW_L', or "SW_H' depending on the following decision rule:
    # 1. If degree distribution is 'power law' then label it 'PA'
    #    * visualize the log-log plot of the graph. If the plot is a negative linear graph, then it is power-law, since
    #          taking log of a power function removes its exponent (e.g. log(x^2) = 2log(x) = 2x', where x' = log(x))
    # 2. If degree distribution is not 'power law' and:
    #    (a) If ACC score and ASP score is both small, label it as 'SW_H'
    #    (b) If ACC score and ASP score is both large, then label it as 'SW_L'

    #import matplotlib.pyplot as plt

    # degree distribution of figures.

    for x in range(len(P1_Graphs)):
        degrees = P1_Graphs[x].degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(P1_Graphs[x])) for i in degree_values]
        #plt.figure()
        #plt.plot(degree_values,histogram, 'o')
        #plt.xscale('log')
        #plt.yscale('log')
        #plt.show()

    # Based on degree distribution, graphs 1 and 4 are power law distributions, while graphs 2,3, and 5 are not.
    graph1 = 'PA'
    graph4 = 'PA'

    # For graphs 2,3, and 5, check their ACC and ASP scores.
    graph2 = P1_Graphs[1]
    graph3 = P1_Graphs[2]
    graph5 = P1_Graphs[4]
    g2_score = (nx.average_clustering(graph2), nx.average_shortest_path_length(graph2))
    g3_score = (nx.average_clustering(graph3), nx.average_shortest_path_length(graph3))
    g5_score = (nx.average_clustering(graph5), nx.average_shortest_path_length(graph5))
    graph2 = 'SW_L'
    graph3 = 'SW_L'
    graph5 = 'SW_H'
    
    return [graph1, graph2, graph3, graph4, graph5]

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    
    # First, create a panda dataframe of the graph which contains the features of the graphs in terms of nodes. 
    # This dataframe should have 1005 rows for the 1005 nodes, and three features: degree of node, clustering score,
    # Department attribute, and Management Salary attribute.

    df = pd.DataFrame(index=G.nodes())
    #df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['Management Salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())

    # May need to use one hot encoding for department values, since they are categorical values, they are not ordinal. 
    #df= pd.get_dummies(df, columns = ['Department'])

    # Now partition the set into train and test subsets, based the presence of valid 'ManagementSalary' values. I.e. Train 
    # set is all rows that have numeric values for 'ManagementSalary' and test set is all other rows (that have NaN values for 
    # 'ManagementSalary').
    
    df_train = df[df['Management Salary'].notnull()]
    df_test = df[df['Management Salary'].isnull()]
    df_test = df_test.drop('Management Salary',axis =1)

    # Perform a train-test split on df_train to get train set and validation set. 

    from sklearn.model_selection import train_test_split

    feature_names = list(df_train.columns[df_train.columns != 'Management Salary'])
    X_df_train = df_train[feature_names]
    y_df_train = df_train['Management Salary']

    X_train, X_val, y_train, y_val = train_test_split(X_df_train, y_df_train, random_state=0)

    # For classifiers which use arithmetic operation between feature variables to calculate decision boundaries, it is necessary
    # to perform feature normalization so that comparison between values of different features makes sense. 

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()

    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val.values), columns=X_val.columns, index=X_val.index)

    # Use Random Forest classifier

    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier(random_state = 0, max_depth = 4)
    clf.fit(X_train_scaled, y_train)

    # Cross-validation of the model for the evaluation metric = AUC.

    from sklearn.model_selection import cross_val_score
    cv_scores = cross_val_score(clf, X_val_scaled, y_val, cv = 10, scoring='roc_auc')
    cv_scores.mean()

    # Predict probability of binary class using RF classifier on test set.

    df_test_scaled = pd.DataFrame(scaler.transform(df_test.values), columns=df_test.columns, index=df_test.index)
    predicted_probabilities = clf.predict_proba(df_test_scaled)

    single_value_prediction = []
    for x in predicted_probabilities:
        if x[0] > x[1]: 
            single_value_prediction.append(1 - x[0])
        else: 
            single_value_prediction.append(x[1])

    df_test_scaled['Management Salary'] = single_value_prediction
    
    return df_test_scaled['Management Salary']

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():
    
    # Extract edge features that are the different measures of the closeness/connectivity/affinity between two nodes. 
    # m1: Common Neighbors m2: Jaccard Coefficient, m3: Resource Allocation, m4: Adamic-Adar Index, m5:  Preferential Attachment
    # (omit community based measures since assigning community of nodes increases the complexity of this assignment)

    future_connections['Common Neighbors'] = [len(list(nx.common_neighbors(G,e[0],e[1]))) for e in future_connections.index]
    future_connections['Jaccard Coefficient'] = [x[2] for x in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [x[2] for x in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic-Adar Index'] = [x[2] for x in nx.adamic_adar_index(G, future_connections.index)]
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]

    # Now partition the dataframe into train and test set, by grouping rows that have 'Future Connection' column values with float 
    # as train set and grouping rows with NaN values as test set.

    df_train = future_connections[future_connections['Future Connection'].notnull()]
    df_test = future_connections[future_connections['Future Connection'].isnull()]
    df_test = df_test.drop('Future Connection',axis =1)

    # Perform a train-test split on df_train to get train set and validation set. 

    from sklearn.model_selection import train_test_split

    feature_names = list(df_train.columns[df_train.columns != 'Future Connection'])
    X_df_train = df_train[feature_names]
    y_df_train = df_train['Future Connection']

    X_train, X_val, y_train, y_val = train_test_split(X_df_train, y_df_train, random_state=0)

    # For classifiers which use arithmetic operation between feature variables to calculate decision boundaries, it is necessary
    # to perform feature normalization so that comparison between values of different features makes sense. 

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()

    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val.values), columns=X_val.columns, index=X_val.index)

    # Use Random Forest classifier

    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier(random_state = 0, max_depth = 4)
    clf.fit(X_train_scaled, y_train)

    # Cross-validation of the model for the evaluation metric = AUC.

    from sklearn.model_selection import cross_val_score
    cv_scores = cross_val_score(clf, X_val_scaled, y_val, cv = 10, scoring='roc_auc')
    cv_scores.mean()

    # Predict probability of binary class using RF classifier on test set.

    df_test_scaled = pd.DataFrame(scaler.transform(df_test.values), columns=df_test.columns, index=df_test.index)
    predicted_probabilities = clf.predict_proba(df_test_scaled)

    single_value_prediction = []
    for x in predicted_probabilities:
        if x[0] > x[1]: 
            single_value_prediction.append(1 - x[0])
        else: 
            single_value_prediction.append(x[1])

    df_test_scaled['Future Connection'] = single_value_prediction
    
    return df_test_scaled['Future Connection']